<a href="https://colab.research.google.com/github/namhawon/2024_Study02/blob/main/14_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4_%EC%B5%9C%EC%8B%A0_%EB%8F%99%ED%96%A5_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPT API로 데이터 전처리하기**

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI  # OpenAI 패키지 임포트

# OpenAI 클라이언트 인스턴스 생성
client = OpenAI(api_key="YOUR_API_KEY")  # 본인의 API 키를 입력하세요

In [ ]:
# GPT-4o-mini 모델을 사용하여 전화번호 전처리
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "다음 전화번호를 표준 형식(예: 010-1234-5678)으로 변환해줘. 표준 형식만 바로 답해줘."},
        {"role": "user", "content": "01061720104"}
    ]
)
data = completion.choices[0].message  # 첫 번째 응답의 메시지 객체 추출
print(data.content)

010-6172-0104


In [ ]:
import openai
import pandas as pd
from tqdm import tqdm

# 전화번호 데이터를 엑셀 파일에서 불러오기
file_path = 'phone_numbers.xlsx'
df = pd.read_excel(file_path)

# 전화번호 전처리 함수 정의
def preprocess_phone_number(phone_number):
    try:
        # GPT-4o-mini 모델을 사용하여 전화번호 전처리
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "다음 전화번호를 표준 형식(예: 010-1234-5678)으로 변환해줘. 표준 형식만 바로 답해줘."},
                {"role": "user", "content": phone_number}
            ]
        )
        data = completion.choices[0].message  # 첫 번째 응답의 메시지 객체 추출
        processed_number = data.content  # 메시지 객체의 내용 추출

        return processed_number
    except:
        return None

# 전화번호 데이터를 전처리하여 새로운 컬럼에 저장 (진행도 표시)
tqdm.pandas()  # tqdm 적용
df['전처리된 전화번호'] = df['전화번호'].progress_apply(preprocess_phone_number)

# 전처리된 데이터를 엑셀 파일로 저장
output_file_path = 'processed_phone_numbers.xlsx'
df.to_excel(output_file_path, index=False)

print(f"전처리된 데이터가 '{output_file_path}' 파일에 저장되었습니다.")

100%|██████████| 100/100 [00:57<00:00,  1.73it/s]

전처리된 데이터가 'processed_phone_numbers.xlsx' 파일에 저장되었습니다.


In [ ]:
df

,전화번호,전처리된 전화번호
0,01061720104,010-6172-0104
1,010.3147.5334,010-3147-5334
2,010 9537 6457,010-9537-6457
3,010 5092 8930,010-5092-8930
4,01060339790,010-6033-9790
...,...,...
95,010-5471-6386,010-5471-6386
96,010 3767 0526,010-3767-0526
97,010-2395-7138,010-2395-7138
98,010.6583.9163,010-6583-9163


# **Sentence Transformer로 문장 임베딩하기**

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import pandas as pd

# 모델 로드
model = SentenceTransformer('bespin-global/klue-sroberta-base-continue-learning-by-mnr')

# 예시 리뷰 문장들
sentences = [
    "이 제품은 정말 훌륭해요! 성능도 좋고 디자인도 예쁩니다.",
    "배송이 너무 늦었어요. 제품 품질은 괜찮지만 서비스가 아쉽네요.",
    "가격 대비 성능이 뛰어나요. 다음에도 구매하고 싶습니다.",
    "상품이 기대했던 것보다 좋지 않습니다. 반품할 예정입니다.",
    "고객 서비스가 매우 친절하고 도움이 많이 되었습니다.",
    "제품이 설명과 다르게 작동하지 않아요. 매우 실망했습니다.",
    "배송도 빠르고 제품도 만족스럽습니다. 추천합니다.",
    "포장이 부실해서 제품이 파손된 채로 왔어요. 다시 보내주세요.",
    "디자인이 세련되고 기능도 다양해서 만족합니다.",
    "사용법이 간단하고 효과가 좋아요. 가족 모두 만족하고 있습니다."
]

# 문장 임베딩 생성
embeddings = model.encode(sentences)

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(embeddings)

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(cosine_similarities, index=sentences, columns=sentences)

In [ ]:
# 각 컬럼에 대해 상위 3개 값을 출력하는 함수 정의
def print_top3_similarities(df):
    for column in df.columns:
        print(f"\n문장: {column}")
        top3 = df[column].nlargest(4).iloc[1:4]  # 자신과의 유사도가 1이기 때문에 제외
        for idx, value in top3.items():
            print(f"유사한 문장: {idx} (유사도: {value:.4f})")

# 상위 3개 유사도 출력
print_top3_similarities(df)


문장: 이 제품은 정말 훌륭해요! 성능도 좋고 디자인도 예쁩니다.
유사한 문장: 디자인이 세련되고 기능도 다양해서 만족합니다. (유사도: 0.6447)
유사한 문장: 가격 대비 성능이 뛰어나요. 다음에도 구매하고 싶습니다. (유사도: 0.3830)
유사한 문장: 배송도 빠르고 제품도 만족스럽습니다. 추천합니다. (유사도: 0.2683)

문장: 배송이 너무 늦었어요. 제품 품질은 괜찮지만 서비스가 아쉽네요.
유사한 문장: 배송도 빠르고 제품도 만족스럽습니다. 추천합니다. (유사도: 0.5954)
유사한 문장: 상품이 기대했던 것보다 좋지 않습니다. 반품할 예정입니다. (유사도: 0.4715)
유사한 문장: 제품이 설명과 다르게 작동하지 않아요. 매우 실망했습니다. (유사도: 0.4669)

문장: 가격 대비 성능이 뛰어나요. 다음에도 구매하고 싶습니다.
유사한 문장: 배송도 빠르고 제품도 만족스럽습니다. 추천합니다. (유사도: 0.3886)
유사한 문장: 이 제품은 정말 훌륭해요! 성능도 좋고 디자인도 예쁩니다. (유사도: 0.3830)
유사한 문장: 제품이 설명과 다르게 작동하지 않아요. 매우 실망했습니다. (유사도: 0.2765)

문장: 상품이 기대했던 것보다 좋지 않습니다. 반품할 예정입니다.
유사한 문장: 제품이 설명과 다르게 작동하지 않아요. 매우 실망했습니다. (유사도: 0.6128)
유사한 문장: 배송이 너무 늦었어요. 제품 품질은 괜찮지만 서비스가 아쉽네요. (유사도: 0.4715)
유사한 문장: 포장이 부실해서 제품이 파손된 채로 왔어요. 다시 보내주세요. (유사도: 0.4352)

문장: 고객 서비스가 매우 친절하고 도움이 많이 되었습니다.
유사한 문장: 배송도 빠르고 제품도 만족스럽습니다. 추천합니다. (유사도: 0.3578)
유사한 문장: 배송이 너무 늦었어요. 제품 품질은 괜찮지만 서비스가 아쉽네요. (유사도: 0.2907)
유사한 문장: 가격 대비 성능이 뛰어나요. 다음에도 구매하고 싶습니다. (유사도: 0.2021)

문장: 제품

# **과제**

GPT API나 SentenceTransformer, 혹은 Huggingface Spaces의 모델들을 탐구해보고 보고서를 작성해주세요. 보고서는 GPT API의 기능 활용 사례 탐구, GPT API의 여러 기능 탐구, SentenceTransformer의 기능 활용 사례 탐구, SentenceTransformer 모델들의 성능 실험, Huggingface Spaces 모델들 Use Case 탐구, 사용 후기 등 자유롭게 해주세요.

**채점 기준**

- 분량 (띄어쓰기 포함이나, 합당한 띄어쓰기여야 함.):
  - 0 ~ 100자: 0점
  - 101 ~ 200자: 10점
  - 201 ~ 300자: 20점
  - 301 ~ 500자: 30점
  - 501 ~ 700자: 40점
  - 701 ~ 1000자: 50점
- 탐구 내용: 50점 만점으로 정성 평가